In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
import csv

In [3]:
def data_preprocessing_augmentation(load_dir, save_dir, rotations_degrees = [-8, -4, 0, 4, 8], crops = 5, crop_box_percentage = 0.8, resize_size = (32, 32), to_greyscale = False, save_csv_only = True ):
    """
    Loads Images and performs the following Preprocessing and Augmentation steps using Image module of Pillow library: 
    1. Creates rotations of each image. 
    2. Crops a centered version of each rotated image. 
    3. Crops random sub-images of each centered image.
    4. Resizes the final Images.
    5. Saves the output in a csv file and optionally as .jpg files.
    
    Arguments:
    load_dir - str -- Specifies the directory to load from
    save_dir - str -- Specifies the directory to save to
    rotations_degrees - int list -- A list of rotations in degrees that specifies the rotation of the produced images 
    crops - int -- The number of random sub- crops generated for each image
    crop_box_percentage - float -- The percentage (0-1) of the size of the box of the sub- cropped image w.r.t. the original centered image
    resize_size - int 2-tuple -- Target size of the final images
    to_greyscale - bool -- If set to true, converts the image to greyscale mode
    save_csv_only - bool -- If set to false, also saves the final images in addition to the csv file
    
    Returns: 
    None
    """
    np.random.seed(1)
    csv_file = open(save_dir + "/dataset.csv", "w")
    csv_writer = csv.writer(csv_file)
    
    #### Loads the images
    n= -1 
    fnames = [os.path.join(load_dir, fname) for fname in os.listdir(load_dir) ]
    for fname in fnames:
        image = Image.open(fname)
        n += 1
        
        #### Getting Rotations of images
        image_rotations = {}
        for rotation in rotations_degrees:
            image_rotations[str(rotation)] = image.rotate(rotation)
            
            #### Cropping centered images
            (width, height) = image.size
            crop_box = ( width * 1.5 // 5., height * 0.8 // 4., width * 3.5 // 5., height * 3. // 4.)
            image_crop = image_rotations[str(rotation)].crop( box = crop_box )
    
            #### Cropping Sub- Images
            (width, height) = image_crop.size
            box_percentage = crop_box_percentage
            offset_max_width = int( ( 1 - box_percentage) * width )
            offset_max_height  = int ( ( 1 - box_percentage) * height )

            image_sub_crops = {}
            for i in range(crops):
                offset_w = np.random.rand() * offset_max_width
                offset_h = np.random.rand() * offset_max_height
                crop_box = ( offset_w, offset_h, box_percentage * width + offset_w, box_percentage * height + offset_h )
                image_sub_crops["crop_" + str(i)] = image_crop.crop(box = crop_box ) 
                
                #### Resize Images and optionally converts to greyscale
                target_size = resize_size
                image_resized = image_sub_crops["crop_" + str(i)].resize( size = target_size)
                if (to_greyscale == True):
                    image_resized = image_resized.convert('L')
                
                ### Save Image
                multiplier_bool = int(n > 239 )
                label = (n - 240 * multiplier_bool ) // 10 
                index = n % 10 + 10 * multiplier_bool #derived from which of 20 original images
            
                if ( save_csv_only == False):
                    image_resized.save(save_dir+ "/"+ str(label)+ "_"+ str(index)+ "_rotation"+ str(rotation)+ "_crop"+ str(i)+ ".jpg")
                
                image_array_flattened = np.array(image_resized, dtype = str).flatten()
                label_image_array = np.insert( image_array_flattened, 0, str(label) )
                csv_writer.writerow(label_image_array)
    csv_file.close()
    return 

In [5]:
#### Performing the Data Preprocessing and Augmentation of the Sign Language Dataset
data_preprocessing_augmentation("Greek_Sign_Language_Dataset_Raw_Images", "Greek_Sign_Language_Dataset_Augmented", rotations_degrees = [-8, -4, 0, 4, 8], crops = 5, crop_box_percentage = 0.85, resize_size = (32, 32), to_greyscale = False, save_csv_only = False)

In [4]:
data_preprocessing_augmentation("Greek_Sign_Language_Dataset_Raw_Images", "Greek_Sign_Language_Dataset_Augmented_Greyscale", rotations_degrees = [-8, -4, 0, 4, 8], crops = 5, crop_box_percentage = 0.85, resize_size = (32, 32), to_greyscale = True, save_csv_only = True )

PermissionError: [Errno 13] Permission denied